# flats-in-cracow data wrangling

## Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import unidecode

from collections import Counter
from IPython.display import display
from sklearn.impute import KNNImputer
from pylab import rcParams
from pathlib import Path

## Setup

In [2]:
# Create directory for images 
Path("img").mkdir(parents=True, exist_ok=True)

# Set default figure size
rcParams['figure.figsize'] = (4, 4)

# Tell pandas how to display floats
pd.options.display.float_format = "{:,.2f}".format

## Goal

I scraped listings of properties for sale in Cracow. We would like to create a model to predict flat prices.

## Data source

Data has been scraped from a website with listings. The data has undergone small transformations along the way. The goal of these transformations was to get the data into a usable state not to check it's validity.

## Data loading

In [3]:
path = '../flats-data/raw_data.csv'

In [4]:
data = pd.read_csv(path, lineterminator='\n')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61307 entries, 0 to 61306
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         61137 non-null  object 
 1   City         47144 non-null  object 
 2   District     33916 non-null  object 
 3   Amount       61077 non-null  float64
 4   Currency     61077 non-null  object 
 5   Property     60720 non-null  object 
 6   Seller       60970 non-null  object 
 7   Area         60819 non-null  float64
 8   Rooms        60102 non-null  float64
 9   Bathrooms    39187 non-null  float64
 10  Parking      26408 non-null  object 
 11  Garden       61307 non-null  bool   
 12  Balcony      61307 non-null  bool   
 13  Terrace      61307 non-null  bool   
 14  Basement     61307 non-null  bool   
 15  New          61307 non-null  bool   
 16  Block        61307 non-null  bool   
 17  Townhouse    61307 non-null  bool   
 18  Apartment    61307 non-null  bool   
 19  Bus 

First we sort the data in from newest to oldest, forcing rows with missing `Date` values to be last.

In [6]:
data = data.sort_values(by='Date', 
                        ascending=False, 
                        na_position='last', 
                        ignore_index=True)

Next we assume that the `Title` column uniquely identifies a listing.

In [7]:
data = data.drop_duplicates(['Title'], keep='first')

After this the shape of the data is:

In [8]:
print(data.shape)

(10604, 24)


## Data exploration

We check for missing values that we will have to deal with.

In [9]:
missing = data.isnull().sum(axis=0)
missing.name = 'Missing'
missing = missing.to_frame()
missing = missing[missing['Missing'] > 0]
missing.sort_values('Missing', ascending=False)

,Missing
Parking,6678
District,4427
Bathrooms,4247
Description,1926
City,1722
Rooms,237
Area,130
Seller,88
Property,82
Amount,8


### Check numeric columns

We see that we have 24 columns at our disposal. 
We inspect the numeric columns to see what we are dealing with. 
In the `Amount` column we note there is a property for sale that costs 1PLN, clearly a erroneous value. 
Next we note that the enourmous maximum in the `Amount` column. That is quite a lot of money and could be considered a potential outlier.
The maximum and minimum of the `Area` column also indicate the existance of outliers. These values are clearly too large. The data will need to undergo a filtering process.

In [10]:
data.describe()

,Amount,Area,Rooms,Bathrooms
count,"10,596.00","10,474.00","10,367.00","6,357.00"
mean,"720,622.10",131.45,2.92,1.32
std,"5,110,158.76","3,542.29",1.32,0.63
min,100.00,1.00,1.00,1.00
25%,"395,000.00",43.00,2.00,1.00
50%,"499,200.00",56.00,3.00,1.00
75%,"720,000.00",80.00,4.00,2.00
max,"521,290,000.00","320,000.00",6.00,4.00


### Check binary columns

We inspect the data to see if binary columns are properly populated and check for imbalances.

In [11]:
binary = data.select_dtypes(bool).columns.to_list()

for col in binary:
    tmp = data[[col, 'Amount']]
    tmp = tmp.fillna('NaN')
    tmp = tmp.groupby(col, as_index=False)
    tmp = tmp.count()
    tmp = tmp.rename(columns={'Amount': 'Count'})
    tmp = tmp.sort_values('Count', ascending=False)
    tmp = tmp.reset_index(drop=True)
    display(tmp)

,Garden,Count
0,False,8504
1,True,2100


,Balcony,Count
0,False,6884
1,True,3720


,Terrace,Count
0,False,9340
1,True,1264


,Basement,Count
0,False,9327
1,True,1277


,New,Count
0,False,7159
1,True,3445


,Block,Count
0,False,9272
1,True,1332


,Townhouse,Count
0,False,9690
1,True,914


,Apartment,Count
0,False,9061
1,True,1543


,Bus stops,Count
0,False,9538
1,True,1066


,Studio,Count
0,False,9900
1,True,704


### Check categorical columns

We inspect categorical columns to assert that they contain "valid" values. Most of these columns were generated by a script during the scraping and etl phase of the project.

In [12]:
categorical = data.select_dtypes('object').columns
categorical = categorical.to_list()
omit = ['Title', 'Link', 'Description', 'Date']

for col in categorical:
    if col not in omit:
        tmp = data[['Amount', col]].copy()
        tmp = tmp.fillna('NaN')
        tmp = tmp.groupby(col, as_index=False)
        tmp = tmp.count()
        tmp = tmp.rename(columns={'Amount': 'Count'})
        tmp = tmp.sort_values('Count', ascending=False)
        tmp = tmp.reset_index(drop=True)
        display(tmp)

,City,Count
0,kraków,8882
1,NaN,1722


,District,Count
0,NaN,4427
1,krowodrza,819
2,stare miasto,699
3,podgorze,655
4,nowa huta,462
5,debniki,448
6,bronowice,441
7,pradnik bialy,433
8,pradnik czerwony,329
9,biezanow,323


,Currency,Count
0,pln,10596
1,NaN,8


,Property,Count
0,flat,9127
1,house,1395
2,NaN,82


,Seller,Count
0,realtor,9712
1,owner,804
2,NaN,88


,Parking,Count
0,NaN,6678
1,garage,1545
2,street,1535
3,no parking,652
4,covered,194


## Data cleaning

We assume that if we know the district, the `City` is `kraków`.

In [13]:
mask = (data['City'].isna() == True) & (data['District'].isna() == False)
data.loc[mask, 'City'] = 'kraków'

We extract more `Parking` information from the property description.

In [14]:
def extract_parking(x):
    if ('garaż' in x or 'garaz' in x or 'parking' in x) and 'podziemny' in x:
        return 'covered'
    elif ('garaż' in x or 'garaz' in x) and 'podziemny' not in x:
        return 'garage'
    elif 'parking' in x and 'podziemny' not in x:
        return 'street'
    else:
        return 'no parking'

In [15]:
mask = (data['Parking'].isna() == True) & (data['Description'].isna() == False)
data.loc[mask, ['Parking', 'Description']] = data.loc[mask, 'Description'].apply(extract_parking)

In [16]:
mask = data['Parking'].isna() == True
data.loc[mask, 'Parking'] = 'no parking'

We confirm that we have dealt with all the `NaN`s in the `Parking` column.

In [17]:
print(data['Parking'].isna().sum())

0


In [18]:
text = data[data['Description'].isna() == False].copy()
text = text['Description'].to_list()
text = ' '.join(text)
text = text.split(' ')
text = [x for x in text if x.isalpha()]
text = [x for x in text if len(x) > 3]

### Filtering

Next we filter the data according to these rules:

In [19]:
data = data[data['City'] == 'kraków']
data = data[data['Currency'] == 'pln']
data = data[data['Property'] == 'flat']
data = data[(data['Amount'] >= data['Amount'].quantile(0.025))]
data = data[(data['Amount'] <= data['Amount'].quantile(0.975))]
data = data[(data['Area'] >= data['Area'].quantile(0.01))]
data = data[(data['Area'] <= data['Area'].quantile(0.99))]
data = data[data['District'] != 'unknown']
data = data[data['District'].isna() == False]
data = data[data['Seller'].isna() == False]
data = data[data['Description'].isna() == False]

In [20]:
data = data.reset_index(drop=True)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4659 entries, 0 to 4658
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4659 non-null   object 
 1   City         4659 non-null   object 
 2   District     4659 non-null   object 
 3   Amount       4659 non-null   float64
 4   Currency     4659 non-null   object 
 5   Property     4659 non-null   object 
 6   Seller       4659 non-null   object 
 7   Area         4659 non-null   float64
 8   Rooms        4602 non-null   float64
 9   Bathrooms    2266 non-null   float64
 10  Parking      4659 non-null   object 
 11  Garden       4659 non-null   bool   
 12  Balcony      4659 non-null   bool   
 13  Terrace      4659 non-null   bool   
 14  Basement     4659 non-null   bool   
 15  New          4659 non-null   bool   
 16  Block        4659 non-null   bool   
 17  Townhouse    4659 non-null   bool   
 18  Apartment    4659 non-null   bool   
 19  Bus st

### Impute missing values

The next step is to fill in missing values for numeric columns `Amount` `Area` `Rooms` and `Bathrooms`. We use the `KNNImputer` to accomplish this.

In [22]:
numeric = list(data.select_dtypes('number').columns)

In [23]:
mask = (data['Bathrooms'].isna() == True | data['Rooms'].isna())
missing = data[numeric]

imputer = KNNImputer(n_neighbors=5)
imputer.fit(missing)

missing = imputer.transform(missing)
missing = pd.DataFrame(missing, columns=numeric)

for col in numeric:
    data[col] = missing[col]
    
for col in numeric:
    data[col] = data[col].apply(lambda x: round(x))    

In [24]:
print(data.shape)

(4659, 24)


## Text data

In [25]:
data_text = data[['Amount', 'Description']].copy()
data_text['Description'] = data_text['Description'].apply(unidecode.unidecode)

In [26]:
text = data_text[data_text['Description'].isna() == False].copy()
text = text['Description'].to_list()
text = ' '.join(text)
text = text.split(' ')
text = [x for x in text if x.isalpha()]
text = [x for x in text if len(x) > 3]

In [27]:
unigrams_cnt = Counter(text)

In [28]:
bigrams = [(text[i], text[i+1]) for i in range(0, len(text)-1)]
bigrams_cnt = Counter(bigrams)
# bigrams_cnt.most_common(50)

In [29]:
trigrams = [(text[i], text[i+1], text[i+2]) for i in range(0, len(text)-2)]
trigrams_cnt = Counter(trigrams)
# trigrams_cnt.most_common(50)

In [30]:
for x in unigrams_cnt.most_common(100):
    if x[1] > 25:
        string = x[0]
        name = 'uni_' + x[0]
        data_text[name] = data_text['Description'].str.contains(string).astype(int)

In [31]:
for x in bigrams_cnt.most_common(100):
    if x[1] > 25:
        string = ' '.join(x[0])
        name = 'bi_'+'_'.join(x[0])    
        data_text[name] = data_text['Description'].str.contains(string).astype(int)    

In [32]:
for x in trigrams_cnt.most_common(100):
    if x[1] > 25:
        string = ' '.join(x[0])
        name = 'tri_'+'_'.join(x[0])    
        data_text[name] = data_text['Description'].str.contains(string).astype(int)    

In [33]:
cols = [x for x in data_text.columns if 'uni_' in x or 'bi_' in x or 'tri_' in x]

In [34]:
corrs = data_text[cols].corrwith(data_text['Amount']).to_frame()
corrs = corrs.reset_index()
corrs.columns = ['N-Gram', 'Correlation']
corrs['Correlation'] = np.abs(corrs['Correlation'])
corrs = corrs.sort_values('Correlation', ascending=False)

In [35]:
print(corrs.head(75).to_string())

                                    N-Gram  Correlation
12                             uni_covered         0.11
209      tri_mieszkania_przynalezy_piwnica         0.10
127           bi_przyjemnosc_zaprezentowac         0.09
63                             uni_piwnica         0.09
124                  bi_przynalezy_piwnica         0.08
101               bi_mieszkania_przynalezy         0.08
238   tri_przyjemnosc_zaprezentowac_oferte         0.08
188                bi_zaprezentowac_oferte         0.08
37                               uni_bloku         0.07
240     tri_zaprezentowac_oferte_sprzedazy         0.07
87                           uni_miejskiej         0.07
24                          uni_przynalezy         0.07
291   tri_przystanki_komunikacji_miejskiej         0.07
62                              uni_garazu         0.07
181                    bi_oferte_sprzedazy         0.07
79                              uni_pokoju         0.06
149                     bi_sieci_miejskiej      

## Save data

Verify that there are no `NaN`s in data.

In [36]:
data.isnull().sum().sum()

0

Remove columns that will not be used further.

In [37]:
data = data.drop(['Title', 
                  'Description', 
                  'Link', 
                  'Property', 
                  'City', 
                  'Currency', 
                  'Date'], axis=1)

Take a last peek at the data.

In [38]:
data.head()

,District,Amount,Seller,Area,Rooms,Bathrooms,Parking,Garden,Balcony,Terrace,Basement,New,Block,Townhouse,Apartment,Bus stops,Studio
0,biezanow,350064,realtor,49,3,1,garage,False,True,False,False,True,False,False,False,False,False
1,pradnik czerwony,445920,realtor,46,2,1,garage,False,True,True,False,False,False,False,False,False,False
2,debniki,375000,realtor,52,2,1,street,False,True,False,False,False,False,False,False,False,False
3,biezanow,284700,realtor,39,2,1,garage,False,True,False,False,True,False,False,False,False,False
4,biezanow,381192,realtor,46,2,1,covered,True,False,False,False,True,False,False,True,False,False


In [39]:
data.describe()

,Amount,Area,Rooms,Bathrooms
count,"4,659.00","4,659.00","4,659.00","4,659.00"
mean,"535,288.92",55.90,2.61,1.10
std,"222,190.45",20.23,0.99,0.33
min,"212,000.00",22.00,1.00,1.00
25%,"390,000.00",41.00,2.00,1.00
50%,"470,000.00",53.00,3.00,1.00
75%,"616,000.00",66.00,3.00,1.00
max,"1,525,000.00",132.00,6.00,4.00


Save it for further analysis.

In [40]:
data.to_csv('../flats-data/cleaned_data.csv', index=False)